In [11]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [12]:
import numpy as np
import pandas as pd
import random

In [13]:
root_path = '/content/drive/MyDrive/BPR/'

## 데이터

### 적재

In [14]:
df = pd.read_csv(root_path + "philadelphia_checkin_small.csv")
df.head()

,newUserId,newBusinessId
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [15]:
print("체크인 수:", len(df))

# 사용자 수 출력
num_users = df["newUserId"].nunique()
print("사용자 수:", num_users)

# 음식점 수 출력
num_businesses = df["newBusinessId"].nunique()
print("음식점 수:", num_businesses)

체크인 수: 332979
사용자 수: 12062
음식점 수: 7011


### 훈련/테스트 셋 분리

In [16]:
# 각 사용자별로 n번째 음식점 방문기록 까지 데이터를 테스트 데이터로, 나머지를 트레이닝 데이터로 분리하는 함수
def split_train_test_data(df, n=3, user_col='newUserId', item_col='newBusinessId', output_dir='.'):
    train_data = []
    test_data = []

    # 사용자별로 그룹화하여 처리
    for user_id, group in df.groupby(user_col):
        # 첫 번째부터 n번째 음식점 방문 기록을 test_data로 추가
        test_data.extend(group.iloc[:n].values.tolist())

        # n+1번째부터 마지막 음식점 방문 기록을 train_data로 추가
        train_data.extend(group.iloc[n:].values.tolist())

    train_df = pd.DataFrame(train_data, columns=[user_col, item_col])
    test_df = pd.DataFrame(test_data, columns=[user_col, item_col])


    train_size = train_df.shape
    test_size = test_df.shape
    print("Train 데이터 크기:", train_size)
    print("Test 데이터 크기:", test_size)
    print("Test 데이터 크기 / Train 데이터 크기 비율:", test_size[0] / train_size[0]) # 전체 크기의 대략 10%

    # CSV 파일로 저장
    train_df.to_csv(output_dir + "train.csv", index=False, header=False, sep='\t')
    test_df.to_csv(output_dir + "test.csv", index=False, header=False, sep='\t')

    print("'train.csv'와 'test.csv' 파일이 생성되었습니다.")

split_train_test_data(df, n=3, output_dir=root_path)

Train 데이터 크기: (296794, 2)
Test 데이터 크기: (36185, 2)
Test 데이터 크기 / Train 데이터 크기 비율: 0.12191958058451316
'train.csv'와 'test.csv' 파일이 생성되었습니다.


### 사용자별 미방문(부정적) 아이템셋 생성

In [17]:
# 각 사용자별로 부정적인 조합 생성
def generate_negative_samples(df, n=100, user_col='newUserId', item_col='newBusinessId', output_dir='.'):
    # 사용자와 음식점 집합 구하기
    users = df[user_col].unique()
    businesses = df[item_col].unique()

    # 사용자별로 부정적인 조합 저장
    user_negative_samples = {}

    # 각 사용자에 대해 부정적인 조합 생성
    for user in users:
        # 사용자가 방문한 음식점 집합
        user_visited_businesses = set(df[df[user_col] == user][item_col].unique())

        # 사용자가 방문하지 않은 음식점 찾기
        user_not_visited_businesses = list(set(businesses) - user_visited_businesses)

        # 랜덤 샘플링하여 부정적인 조합 생성
        if len(user_not_visited_businesses) > n: #n개 이상이면 데이터가 많아지니 제한을 둠
            user_negative_samples[user] = random.sample(user_not_visited_businesses, n)
        else:
            user_negative_samples[user] = user_not_visited_businesses

    # 부정적인 조합을 CSV 파일로 저장
    with open(output_dir + "negative.csv", 'w') as f:
        for user, negatives in user_negative_samples.items():
            f.write(f"{user},{','.join(map(str, negatives))}\n")

    print("파일 'negative.csv'가 생성되었습니다.")

generate_negative_samples(df, n=100, output_dir=root_path)

파일 'negative.csv'가 생성되었습니다.


### 테스트셋의 미방문 아이템셋 생성

In [18]:
# 개인별로 방문하지 않은 음식점 샘플링
def generate_individual_negative_sample(df, user_id, n=100, user_col='newUserId', item_col='newBusinessId'):
    # 모든 음식점 목록 가져오기
    businesses = df[item_col].unique()
    # 사용자가 방문한 음식점 목록 가져오기
    user_visited_businesses = set(df[df[user_col] == user_id][item_col].unique())
    # 사용자가 방문하지 않은 음식점 목록 계산
    user_not_visited_businesses = list(set(businesses) - user_visited_businesses)
    # 방문하지 않은 음식점 중에서 n개를 랜덤으로 샘플링
    if len(user_not_visited_businesses) > n:
        random_negative = random.sample(user_not_visited_businesses, n)
    else:
        random_negative = user_not_visited_businesses
    return random_negative


# testData를 위한 부정적인 음식점 샘플링
with open(root_path + 'negative.csv', 'r') as negative_file \
  , open(root_path + 'test.csv', 'r') as test_file \
  , open(root_path + 'test_negative.csv', 'w') as test_negative_file:
    read_negative = negative_file.readlines()
    read_test = test_file.readlines()
    print(len(read_negative))  # 12062 -> 사용자 수만큼 개인별로 방문하지 않은 음식점 집합 완성

    for i in range(len(read_negative)):  # 각 사용자에 대해
        negative = read_negative[i]  # 사용자 i에 대한 방문하지 않은 음식점 집합 가져오기
        user_id, user_id_negative_businesses = negative.strip().split(',', 1)  # 사용자 ID와 방문하지 않은 음식점 구분
        #user_id_negative_businesses = user_id_negative_businesses.split(',')  # 방문하지 않은 음식점들을 리스트로 분리

        for j in range(i * 3, (i + 1) * 3):  # 각 사용자의 3개의 음식점 방문기록
            test = read_test[j % len(read_test)]  # 테스트 데이터 중 해당 사용자의 음식점 방문기록
            user_trip = test.strip()  # 공백 제거

            # 개별 사용자의 부정적인 음식점 생성
            individual_negative_sample = generate_individual_negative_sample(df, int(user_id), n=100)

            # 테스트 데이터와 부정적인 샘플을 결합하여 git에 있는 BPR모델 입력에 맞게 변환
            test_negative_business = user_trip + '\t' + '\t'.join(map(str, individual_negative_sample)) + '\n'
            test_negative_file.write(test_negative_business)

print("파일 'test_negative.csv'가 생성되었습니다.")

12062
파일 'test_negative.csv'가 생성되었습니다.
